In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
import sys
sys.path.append('/home/chenani/ownCloud/Workspaces/Eclipse/dataAnalysis/Sleep-current/src/')

In [3]:
%matplotlib 
#inline
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as pl
import numpy as np
import pickle as pkl
import os
import seaborn as sns
import pandas as pd
import fnmatch
from scipy.stats import binom_test
from scipy.io import loadmat,savemat

Using matplotlib backend: Qt4Agg


/home/chenani/anaconda/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [4]:
def locate(pattern, root=os.curdir):
    '''Locate all files matching supplied filename pattern in and below
        supplied root directory.
    '''
    for path, dirs, files in os.walk(os.path.abspath(root)):
        for filename in fnmatch.filter(files, pattern):
            yield [path,filename]
def findClosestPoint(values,array):
    values = np.array(values)
    array = np.array(array)
    idx = np.array([np.square(array - value).argmin() for value in values])
    return idx,array[idx]
def sigCounter(df,min_N = 3,pSig = 5e-2):
    dbMinN = df[(df.lSub>min_N)|(df.rSub>min_N)]
    nT = dbMinN.shape[0]
    nS = dbMinN[(dbMinN.Lpv<0.05)|(dbMinN.Rpv<0.05)].shape[0]
    print nT,nS,binom_test(nS,nT,pSig),1e2*nS/nT
    return 1e2*nS/nT,binom_test(nS,nT,pSig)
sns.set_style('white')

In [313]:
dataFolder = '/home/chenani/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/control/Rat3661Control/'
tdat = loadmat('/home/chenani/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/files/replay_DataSet/rat3661.postsleepDay1.mat')['tdat']
medat = loadmat('/home/chenani/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/files/replay_DataSet/control/Rat3661-D1-sleep02.mat')

In [32]:
asleepDBs = sorted([os.path.join(item[0],item[1])for item in locate('*pop*.db',dataFolder)])
awakeDBs  = sorted([os.path.join(item[0],item[1])for item in locate('*awake*.db',dataFolder)])
asleepDBs,awakeDBs

(['/home/chenani/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/control/Rat3661Control/dataFiles/sleepDB/3661-popDB-D1.db',
  '/home/chenani/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/control/Rat3661Control/dataFiles/sleepDB/3661-popDB-D2.db',
  '/home/chenani/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/control/Rat3661Control/dataFiles/sleepDB/3661-popDB-D3.db'],
 ['/home/chenani/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/control/Rat3661Control/dataFiles/awakeReplays-3661-D1-PR.db',
  '/home/chenani/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/control/Rat3661Control/dataFiles/awakeReplays-3661-D1-RO.db',
  '/home/chenani/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/control/Rat3661Control/dataFiles/awakeReplays-3661-D1.db',
  '/home/chenani/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/control/Rat3661Control/dataFiles/awakeReplays-3661-D2-PR.db',
  '/home/chenani/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/control/Rat3661Control/dataFil

In [35]:
binwidth = 0.1
A = np.linspace(0,1.,1.0/binwidth+1)
binwidth = np.diff(A)[0]
corrBinsCenters = np.union1d(-1*A,A)
corrBins = corrBinsCenters-binwidth/2
corrBins = np.insert(corrBins,[corrBins.size],[corrBins[-1]+binwidth])
##########################################
SigPrcntMin = 90
sigN = []
sigPrcnt = []
SigCorrMin = []
for n in range(4,11):
    crr = pkl.load(open('/home/chenani/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/RankOrder/corr'+str(n)+'.crr','rb'))
    absCrr = np.abs(crr)
    cSet = np.unique(absCrr)
    prcSet = 100.0*np.cumsum([(absCrr==cor).sum() for cor in cSet])/ np.math.factorial(n)
    #print n,prcSet
    SigCorrMin.append(cSet[np.where(prcSet>= SigPrcntMin)[0][0]])
#################################
for dbFile in asleepDBs[:1]:
    asleepDF = pd.read_pickle(dbFile)
    print asleepDF.date[0]
    for sess in ['sleep01','sleep02']:
        df = asleepDF[asleepDF.session==sess]
        for idx,row in df.iterrows():
            if row.lSub>3:
                if np.abs(row.Lcorr) > SigCorrMin[min(int(row.lSub - 4),len(SigCorrMin)-1)]:
                    asleepDF.Lpv[idx]-=0.96
                    #print row.lSub,row.Lcorr,SigCorrMin[min(int(row.lSub - 4),len(SigCorrMin)-1)]
            if row.rSub>3:
                if np.abs(row.Rcorr) > SigCorrMin[min(int(row.rSub - 4),len(SigCorrMin)-1)]:
                    asleepDF.Rpv[idx]-=0.96
#    asleepDF.to_pickle(dbFile.split('-')[0]+'-roDB-'+ dbFile.split('-')[-1])
            

/home/chenani/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: DeprecationWarning: object of type <type 'float'> cannot be safely interpreted as an integer.
  from ipykernel import kernelapp as app
/home/chenani/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


03-03-15Day1


/home/chenani/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [59]:
SigPrcntMin = 95
for n in range(4,11):
    crr = pkl.load(open('/home/chenani/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/RankOrder/corr'+str(n)+'.crr','rb'))
    absCrr = np.abs(crr)
    cSet = np.unique(absCrr)
    prcSet = 100.0*np.cumsum([(absCrr==cor).sum() for cor in cSet])/ np.math.factorial(n)
    pl.plot(cSet[prcSet>SigPrcntMin],prcSet[prcSet>SigPrcntMin],lw=3,label=n)
pl.legend()

In [7]:
pl.plot(range(4,15),[sigCounter(asleepDF,min_N=ii,pSig=0.1) for ii in range(4,15)],'v')

1110 159 4.95575218117e-06 14.3243243243
664 103 9.62502578382e-06 15.5120481928
359 73 5.40532109528e-09 20.3342618384
210 45 8.28593690444e-07 21.4285714286
139 32 6.28759199991e-06 23.0215827338
82 21 4.23954639203e-05 25.6097560976
57 15 0.000368319429784 26.3157894737
38 13 4.82132093716e-05 34.2105263158
16 6 0.00329675144067 37.5
6 1 0.468559 16.6666666667
3 0 1.0 0.0


In [25]:
f,ax = pl.subplots(1,1)
for ii in range(4,7):
    if sigCounter(asleepDF,min_N=ii,pSig=0.1)[1]<5e-2:
        ax.plot(ii,sigCounter(asleepDF,min_N=ii,pSig=0.1)[0],'v',c='r')
        print ii
    else:
        ax.plot(ii,sigCounter(asleepDF,min_N=ii,pSig=0.1)[0],'v',c='k')

46 6 0.45870941883 13.0434782609
46 6 0.45870941883 13.0434782609
10 2 0.2639010709 20.0
10 2 0.2639010709 20.0
2 0 1.0 0.0
2 0 1.0 0.0


In [274]:
lc = asleepDF[(asleepDF.session=='sleep02')&(asleepDF.lSub>4)].Lcorr
rc = asleepDF[(asleepDF.session=='sleep02')&(asleepDF.rSub>4)].Rcorr

In [49]:
cdat = tdat[0][0][0]

In [77]:
pl.hist([cdat[0],lc,rc],corrBins,color=['r','0.8','0.7'],label=['c','l','r'],normed=True)
pl.legend()

In [282]:
s2 = asleepDF[asleepDF.session=='sleep02']
lc.size+rc.size,-np.intersect1d(lc.index,rc.index).size,cdat[0].size,df[df.session=='sleep02'].shape

(511, -203, 414, (308, 15))

In [267]:
pl.plot(lc,np.ones(lc.size),'d')
#pl.plot(rc,np.ones(rc.size)-1e-2,'d')
pl.plot(cdat[0],np.ones(cdat[0].size)+1e-2,'d')
pl.ylim(0.65,1.35)

(0.65, 1.35)

In [88]:
surrDists = loadmat('/home/chenani/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/files/replay_DataSet/rt2.mat')['rt2'][0]

In [246]:
f,axarr = pl.subplots(2,3,figsize=[12,8])
for n in range(4,10):
    crr = pkl.load(open('/home/chenani/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/RankOrder/corr'+str(n)+'.crr','rb'))
    srr = surrDists[n-4][1][0].T
    axarr[(n-4)/3,np.mod(n-4,3)].hist([srr,crr],corrBins,color=['k','r'],label=['Christian','Ali'],normed=True)
pl.legend()

In [312]:
s2[(s2.rSub>4)|(s2.lSub>4)].shape[0]+s2[(s2.rSub>4)^(s2.lSub>4)].shape[0]
#s24 = s2[(~np.isnan(s2.Lcorr))|(~np.isnan(s2.Rcorr))]#.shape[0]#-s2[(~np.isnan(s2.Lcorr))^(~np.isnan(s2.Rcorr))].shape[0]
#s24[(s24.lSub>4)|(s24.rSub>4)].shape

413

In [300]:
s2[(s2.rSub>4)^(s2.lSub>4)]

,t_start,t_end,peak_value,t_peak,Lcorr,Rcorr,Lpv,Rpv,tSub,lSub,rSub,epoch,date,session,animal
2808,5484.1845,5484.3465,0.005851,5484.2195,-0.700000,-0.2,1.00,1.00,5.0,5.0,4.0,rem,03-03-15Day1,sleep02,Rat3661
2812,5486.1985,5486.2835,0.003685,5486.2525,0.000000,NaN,1.00,1.00,5.0,5.0,3.0,rem,03-03-15Day1,sleep02,Rat3661
2859,5520.4455,5520.5155,0.008588,5520.4815,-0.300000,0.0,1.00,1.00,5.0,5.0,4.0,rem,03-03-15Day1,sleep02,Rat3661
2864,5524.0575,5524.1565,0.009956,5524.1085,-0.357143,-0.8,1.00,1.00,8.0,7.0,4.0,rem,03-03-15Day1,sleep02,Rat3661
2871,5528.3145,5528.4525,0.005077,5528.3825,-0.657143,0.4,1.00,1.00,6.0,6.0,4.0,rem,03-03-15Day1,sleep02,Rat3661
2924,5567.0215,5567.3035,0.005126,5567.1545,-0.400000,-0.7,1.00,1.00,5.0,4.0,5.0,NA,03-03-15Day1,sleep02,Rat3661
2957,5605.0535,5605.1285,0.006512,5605.0905,0.200000,0.0,1.00,1.00,5.0,4.0,5.0,rem,03-03-15Day1,sleep02,Rat3661
2981,5637.6875,5637.7725,0.004816,5637.7515,0.600000,NaN,1.00,1.00,5.0,5.0,3.0,rem,03-03-15Day1,sleep02,Rat3661
3009,5663.0035,5663.0555,0.005386,5663.0265,0.400000,NaN,1.00,1.00,5.0,5.0,3.0,NA,03-03-15Day1,sleep02,Rat3661
3025,5684.3245,5684.5435,0.004355,5684.4505,-0.400000,0.7,1.00,1.00,5.0,4.0,5.0,rem,03-03-15Day1,sleep02,Rat3661
